# Hadoop Word Count


In [1]:
%%writefile WordCount.java
import java.io.IOException;
import java.util.StringTokenizer;

import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.Mapper;
import org.apache.hadoop.mapreduce.Reducer;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;

public class WordCount {

  public static class TokenizerMapper
       extends Mapper<Object, Text, Text, IntWritable>{

    private final static IntWritable one = new IntWritable(1);
    private Text word = new Text();

    public void map(Object key, Text value, Context context
                    ) throws IOException, InterruptedException {
      StringTokenizer itr = new StringTokenizer(value.toString());
      while (itr.hasMoreTokens()) {
        word.set(itr.nextToken());
        context.write(word, one);
      }
    }
  }

  public static class IntSumReducer
       extends Reducer<Text,IntWritable,Text,IntWritable> {
    private IntWritable result = new IntWritable();

    public void reduce(Text key, Iterable<IntWritable> values,
                       Context context
                       ) throws IOException, InterruptedException {
      int sum = 0;
      for (IntWritable val : values) {
        sum += val.get();
      }
      result.set(sum);
      context.write(key, result);
    }
  }

  public static void main(String[] args) throws Exception {
    Configuration conf = new Configuration();
    Job job = Job.getInstance(conf, "word count");
    job.setJarByClass(WordCount.class);
    job.setMapperClass(TokenizerMapper.class);
    job.setCombinerClass(IntSumReducer.class);
    job.setReducerClass(IntSumReducer.class);
    job.setOutputKeyClass(Text.class);
    job.setOutputValueClass(IntWritable.class);
    FileInputFormat.addInputPath(job, new Path(args[0]));
    FileOutputFormat.setOutputPath(job, new Path(args[1]));
    System.exit(job.waitForCompletion(true) ? 0 : 1);
  }
}

Writing WordCount.java


In [2]:
%env PATH=/usr/lib/jvm/java-8-openjdk-amd64/bin:/home/ubuntu/bin:/home/ubuntu/bin:/home/ubuntu/miniconda3/bin:/home/ubuntu/miniconda3/condabin:/usr/lib/jvm/java-8-openjdk-amd64/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin
%env HADOOP_CLASSPATH=/usr/lib/jvm/java-8-openjdk-amd64/lib/tools.jar

env: PATH=/usr/lib/jvm/java-8-openjdk-amd64/bin:/home/ubuntu/bin:/home/ubuntu/bin:/home/ubuntu/miniconda3/bin:/home/ubuntu/miniconda3/condabin:/usr/lib/jvm/java-8-openjdk-amd64/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin
env: HADOOP_CLASSPATH=/usr/lib/jvm/java-8-openjdk-amd64/lib/tools.jar


You probably have `HADOOP_HOME` set up during the installation instructions.
If that's not the case, please configure it.

In [3]:
%env HADOOP_HOME=/home/ubuntu/hadoop-3.3.1

env: HADOOP_HOME=/home/ubuntu/hadoop-3.3.1


### Create JAR file

In [4]:
!$HADOOP_HOME/bin/hadoop com.sun.tools.javac.Main WordCount.java

In [5]:
!jar cf wc.jar WordCount*.class

## Launch job

In [6]:
!$HADOOP_HOME/bin/hadoop jar wc.jar WordCount ./data ./out | head

2022-03-17 15:17:52,689 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-03-17 15:17:52,768 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-03-17 15:17:52,768 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2022-03-17 15:17:52,831 WARN mapreduce.JobResourceUploader: Hadoop command-line option parsing not performed. Implement the Tool interface and execute your application with ToolRunner to remedy this.
2022-03-17 15:17:52,894 INFO input.FileInputFormat: Total input files to process : 1
2022-03-17 15:17:52,915 INFO mapreduce.JobSubmitter: number of splits:1
2022-03-17 15:17:53,043 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1965851386_0001
2022-03-17 15:17:53,043 INFO mapreduce.JobSubmitter: Executing with tokens: []
2022-03-17 15:17:53,159 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
2022-03-17 15:17:53,160 INFO mapreduce.Job: Running job: job_local1965851386_

In [7]:
# print wordcout output
!$HADOOP_HOME/bin/hadoop fs -cat ./out/part-r-00000 | head

10	1
America	1
American	1
As	2
But	2
Classification	1
Contents	1
Details	2
Different	1
External	1
